In [1]:
import requests
import json

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import openweathermapy.core as owm

from config import api_key
from config import gkey
from pprint import pprint
import datetime
#import time
#from datetime import timedelta
#from datetime import datetime
#from config import gkey

In [8]:
#Input the target city and state
target_city = input("Enter city & state in the format 'city,state: ")

Enter city & state in the format 'city,state: San Francisco, CA


In [9]:
City = target_city.split(",")[0]

In [10]:
#Google Maps API
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

#Creating the params dict with address and key
params = {"address": target_city, "key": gkey}

In [11]:
# Run request (Build URL using the base_url & params)
response = requests.get(base_url, params=params)
#print(response.url)

# Convert the response to JSON
city_geo = response.json()

# Extract Lat/Lng
lat = city_geo["results"][0]["geometry"]["location"]["lat"]
lng = city_geo["results"][0]["geometry"]["location"]["lng"]

# Print results
print("%s: %s, %s" % (target_city, lat, lng))

#Converting the lat & lng to string
lat = str(lat)
lng = str(lng)

San Francisco, CA: 37.7749295, -122.4194155


In [42]:
#https://developers.google.com/places/web-service/supported_types

#Segregated the places types into indoor and outdoor activities 
Acty_In = pd.read_excel("Resources/Indoor.xlsx")
Acty_Out = pd.read_excel("Resources/Outdoor.xlsx")
Acty_EveIn = pd.read_excel("Resources/EveIndoor.xlsx")
Acty_EveInOut = pd.read_excel("Resources/EveInOut.xlsx")
Acty_NightIn = pd.read_excel("Resources/NightIndoor.xlsx")
Acty_NightInOut = pd.read_excel("Resources/NightInOut.xlsx")

#Converting the dataframe into list
Acty_Outlist = Acty_Out["Outdoor"].tolist()
Acty_Inlist = Acty_In["Indoor"].tolist()
Acty_EveInlist = Acty_EveIn["EveIndoor"].tolist()
Acty_EveInOutlist = Acty_EveInOut["EveInOut"].tolist()
Acty_NightIn = Acty_NightIn["NightIndoor"].tolist()
Acty_NightInOut = Acty_NightInOut["NightInOut"].tolist()

In [29]:
#Read activities CSV
# Indoor_Outdoor_Weather = "Resources/Indoor_Outdoor_Weather - API Target Names.csv"
# activities_df = pd.read_csv(Indoor_Outdoor_Weather, encoding="ISO-8859-1")
# activities_df.head()

#Read all Main weather csv files 
main = "Resources/Indoor_Outdoor_Weather - main.csv"
main_df = pd.read_csv(main, encoding="ISO-8859-1")
Atmosphere = "Resources/Indoor_Outdoor_Weather - Atmosphere.csv"
atmosphere_df = pd.read_csv(Atmosphere, encoding="ISO-8859-1")
Drizzle= "Resources/Indoor_Outdoor_Weather - Drizzle.csv"
drizzle_df = pd.read_csv(Drizzle, encoding="ISO-8859-1")
Rain = "Resources/Indoor_Outdoor_Weather - Rain.csv"
rain_df = pd.read_csv(Rain, encoding="ISO-8859-1")
Snow ="Resources/Indoor_Outdoor_Weather - Snow.csv"
snow_df = pd.read_csv(Snow, encoding="ISO-8859-1")


settings = {"units": "metric", "appid": api_key}

#Get current weather
current_weather = owm.get_current(City, **settings)
#print(f"Current weather object: {current_weather}.")

#Get weather forecast
#forecast_weather = owm.get_forecast_hourly(City, **settings)
#pprint(forecast_weather)

summary = ["name", 'weather', "main", "wind", 'clouds']
data = current_weather(*summary)
#print(f"The current weather summary is : {data}.")

currentmain = data[1][0]["main"]
#print (currentmain)

temp = data[2]['temp']
#print (temp)

humidity = data[2]['humidity']
#print (humidity)

windspeed = data[3]["speed"]
#print(windspeed)

currentdesc = data[1][0]['description']
#print (currentdesc)

#https://en.wikipedia.org/wiki/Beaufort_scale

finalresult=""
 
# loop through main weather types to determine if outdoor or indoor 
if currentmain=="Thunderstorms":
    print(f'Staysafe!!! {currentmain} in the {City}')
    Words_of_advice = "Staysafe!!! "+ currentmain + " in the " + City
    finalresult="indoor"

elif currentmain=="Rain":
    #read rain.df to find which
    for index, desctype in rain_df.iterrows():
        if currentdesc == desctype["Description"]:
            finalresult=rain_df["Rating"][index]
elif currentmain=="Snow" :
    #read snow.df to find which
    for index, desctype in snow_df.iterrows():
        if currentdesc == desctype["Description"]:
            finalresult=desctype["Rating"][index]

elif currentmain=="Drizzle" :
    #read drizzledf to find which
    for index, desctype in drizzle_df.iterrrows():
        if currentdesc == desctype["Description"]:
            finalresult=drizzle_df["Rating"][index]

elif currentmain=="Atmosphere" :
    #read atmospheredf to find which
    for index, desctype in atmosphere_df.iterrows():
        if currentdesc== desctype["Description"]:
            finalresult=atmosphere_df["Rating"][index]

elif(currentmain=="Clouds") or (currentmain=="Clear") or (currentmain=="Mist"):
    if  (temp<7 or temp>35):
        print(f'Better to be safe inside as it is extreme temperature {temp} outside in {City}')
        finalresult="indoor"
    elif(windspeed>38):
        #https://en.wikipedia.org/wiki/Beaufort_scale
        print(f'Better to be safe inside as it windy ({windspeed}) outside in {City}')
        finalresult="indoor"
    elif(humidity>60): 
        print(f'Better to be inside as it is quite humid ({humidity}) outside in {City}')
        finalresult="indoor" 
    else:
        print(f'It is safe & pleasant!!! Enjoy outside in the current weather of {currentmain} in {City}')
        Words_of_advice = "It is safe & pleasant!!! Enjoy outside in the current weather of "+ currentmain +" in "+ City
        finalresult="outdoor"
#print (finalresult)

It is safe & pleasant!!! Enjoy outside in the current weather of Clouds in San Francisco


In [30]:
now = datetime.datetime.now()
print(now)
At12= now.replace(hour=12, minute=0, second=0, microsecond=0)
At15 = now.replace(hour=15, minute=0, second=0, microsecond=0)
At18 = now.replace(hour=18, minute=0, second=0, microsecond=0)
At21 = now.replace(hour=21, minute=0, second=0, microsecond=0)

# if (now < At12):
#     print("Plan for the morning")
# elif(now >= At12 and now < At15):
#     print("Plan for the noon")
# elif(now >= At15 and now < At18):
#     print("Plan for the evening")
# else:
#     print("Plan for the night")

2018-07-05 17:51:36.319245


In [43]:
Acty = finalresult
if (Acty == "outdoor"):
    if(now < At12):
        print(Words_of_advice)
        print("It's morning, plan outdoor activities for the entire day")
        Acty = Acty_Outlist
    elif((now >= At12 and now < At15)):
        print(Words_of_advice)
        print("It's afternoon, plan outdoor activities for the day")
        Acty = Acty_Outlist
    elif((now >= At15 and now < At18)):
        print(Words_of_advice)
        print("It's evening, though you can go outdoor,however, we would suggest indoor activities for the evening as parks,zoos might be closed. We would add stadiums though as there might be matches and campgrounds if you are adventurous!!!")
        Acty = Acty_EveInOutlist
    elif((now >= At21)):
        print(Words_of_advice)
        print("It's lovely night, though you can go outdoor,however, we would recommend indoor activities for the night,as parks,zoos are closed. We would add stadiums though as there might be matches and campgrounds if you are adventurous!!!")
        Acty = Acty_NightInOutlist 
elif (Acty == "indoor"):        
    if(now < At12):
        print(Words_of_advice)
        print("It's morning, plan indoor activities for the entire day")
        Acty = Acty_Inlist
    elif((now >= At12 and now < At15)):
        print(Words_of_advice)
        print("It's afternoon, plan indoor activities for the day")
        Acty = Acty_Inlist
    elif((now >= At15 and now < At18)):
        print(Words_of_advice)
        print("It's evening, plan indoor activities for the evening")
        Acty = Acty_EveInlist 
    elif((now >= At21)):
        print(Words_of_advice)
        print("It's night, plan indoor activities for the night")
        Acty = Acty_NightInlist 

It is safe & pleasant!!! Enjoy outside in the current weather of Clouds in San Francisco
It's evening, though you can go outdoor,however, we would suggest indoor activities for the evening as parks,zoos might be closed. We would add stadiums though as there might be matches and campgrounds if you are adventurous!!!


In [33]:
#Hint: See https://developers.google.com/places/web-service/supported_types
# Acty = finalresult

# if (Acty == "outdoor" or Acty == "Outdoor" or Acty == "OUTDOOR"):
#     Acty = Acty_Outlist
# elif (Acty == "indoor"or Acty == "Indoor" or Acty == "INDOOR"):
#     Acty = Acty_Inlist

In [34]:
#Google maps nearby places search API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

places_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"

In [60]:
#Find the names and addresses of nearbyplaces in given city,state entered by the
#user.

type_of_place = []
place_names = []
addresses = []
rating = []
openorclose = []
#Noinfo = []
for i in range(len(Acty)):
    
    target_type = Acty[i]
    #radius = 40000

# Build URL using the Google Maps API and other fields
    #target_url = base_url+"key="+gkey+"&location="+lat+","+lng+"&radius="+str(radius)+"&type="+target_type#+"&opennow"
    target_url = base_url+"key="+gkey+"&location="+lat+","+lng+"&rankby=distance"+"&type="+target_type#+"&opennow"
    #print(target_url)

# Run request
    response = requests.get(target_url)
    city_placestogo = response.json()
    #print(city_placestogo)

    if city_placestogo["status"] == "ZERO_RESULTS":
        print(f'{Acty[i]} : No {Acty[i]} near your place of choice')
        type_of_place.append(target_type)
        place_names.append("Not Available ")
        addresses.append("Not Available ")
        rating.append("Not Available ")
        openorclose.append("Not Available ")
        #print("\n")
    else:
        print(f'{Acty[i].upper()} : Bingo!!! We have found good {Acty[i]}s near your place of choice!!!')
        #print("\n")
        #count = 1
        for item in city_placestogo["results"]:
            myplace = item["name"]
            myplace = myplace.replace(" ", "%20")
            target_url = places_url+"key="+gkey+"&input="+myplace+"&inputtype=textquery"+"&fields=formatted_address,name,rating,opening_hours"
            #print(target_url)
            response = requests.get(target_url)
            data = response.json()
            #print(data)
            type_of_place.append(target_type)
            try:
                place_names.append(data["candidates"][0]["name"])
            except:
                place_names.append(item["name"])
            try:    
                addresses.append(data["candidates"][0]["formatted_address"])
            except:
                addresses.append(item["vicinity"])
            try:    
                rating.append(data["candidates"][0]["rating"])
            except:   
                rating.append(np.nan)
            try:  
                if(data["candidates"][0]["opening_hours"]["open_now"]):
                    openorclose_status = "Open"
                else:
                    openorclose_status = "Closed"
                openorclose.append(openorclose_status)
            except:
                openorclose.append("Check the opening hours on the website")
                #print("No or incomplete information found")
            #count = count + 1
        #print("\n")    

AQUARIUM : Bingo!!! We have found good aquariums near your place of choice!!!
ART_GALLERY : Bingo!!! We have found good art_gallerys near your place of choice!!!
BAKERY : Bingo!!! We have found good bakerys near your place of choice!!!
BAR : Bingo!!! We have found good bars near your place of choice!!!
BOOK_STORE : Bingo!!! We have found good book_stores near your place of choice!!!
BOWLING_ALLEY : Bingo!!! We have found good bowling_alleys near your place of choice!!!
CAFE : Bingo!!! We have found good cafes near your place of choice!!!
CASINO : Bingo!!! We have found good casinos near your place of choice!!!
FLORIST : Bingo!!! We have found good florists near your place of choice!!!
GYM : Bingo!!! We have found good gyms near your place of choice!!!
LIBRARY : Bingo!!! We have found good librarys near your place of choice!!!
LODGING : Bingo!!! We have found good lodgings near your place of choice!!!
MOVIE_RENTAL : Bingo!!! We have found good movie_rentals near your place of choice!!!


In [61]:
Df = pd.DataFrame({"Type":type_of_place,"Place":place_names,"Address":addresses, "Rating":rating,
                  "OpenorClose":openorclose})

In [62]:
Df.columns
Df["Type"].nunique()

21

In [63]:
Df[["Type","Place","Address","Rating","OpenorClose"]]

,Type,Place,Address,Rating,OpenorClose
0,aquarium,Aquarium Services,"500 Terry A Francois Boulevard, San Francisco",NaN,Check the opening hours on the website
1,aquarium,"Pure Aqua, Inc.","2230 S Huron Dr, Santa Ana, CA 92704, USA",4.6,Open
2,aquarium,Aquarium of the Bay,"PIER 39 The Embarcadero &, Beach St, San Franc...",4.0,Open
3,aquarium,California Academy of Sciences,"55 Music Concourse Dr, San Francisco, CA 94118...",4.6,Open
4,aquarium,Steinhart Aquarium at The CA Academy of Sciences,"55 Music Concourse Dr, San Francisco, CA 94118...",4.9,Open
5,aquarium,Fairy Lake Discus Palace,"1609 Clement St, San Francisco, CA 94121, USA",2.9,Open
6,aquarium,ABC Aquatic,"447 26th Ave, San Francisco, CA 94121, USA",5.0,Check the opening hours on the website
7,art_gallery,Gallery 415,"415 Front St, Catasauqua, PA 18032, USA",5.0,Check the opening hours on the website
8,art_gallery,Negative Concord,"50 Brady Street, San Francisco",NaN,Check the opening hours on the website
9,art_gallery,Ian Momsen Art,"50 Brady Street, San Francisco",NaN,Check the opening hours on the website


In [64]:
#Df = Df.reset_index()
#Df = Df.set_index("Type")
#Df[["Type","Place","Address","Rating","OpenorClose"]]
Df.fillna(0, inplace=True)
#Df[Df["OpenorClose"] == "Open"]
# Df1 = Df[Df["Type"]=="bakery"]
# Df1
Df

,Address,OpenorClose,Place,Rating,Type
0,"500 Terry A Francois Boulevard, San Francisco",Check the opening hours on the website,Aquarium Services,0.0,aquarium
1,"2230 S Huron Dr, Santa Ana, CA 92704, USA",Open,"Pure Aqua, Inc.",4.6,aquarium
2,"PIER 39 The Embarcadero &, Beach St, San Franc...",Open,Aquarium of the Bay,4.0,aquarium
3,"55 Music Concourse Dr, San Francisco, CA 94118...",Open,California Academy of Sciences,4.6,aquarium
4,"55 Music Concourse Dr, San Francisco, CA 94118...",Open,Steinhart Aquarium at The CA Academy of Sciences,4.9,aquarium
5,"1609 Clement St, San Francisco, CA 94121, USA",Open,Fairy Lake Discus Palace,2.9,aquarium
6,"447 26th Ave, San Francisco, CA 94121, USA",Check the opening hours on the website,ABC Aquatic,5.0,aquarium
7,"415 Front St, Catasauqua, PA 18032, USA",Check the opening hours on the website,Gallery 415,5.0,art_gallery
8,"50 Brady Street, San Francisco",Check the opening hours on the website,Negative Concord,0.0,art_gallery
9,"50 Brady Street, San Francisco",Check the opening hours on the website,Ian Momsen Art,0.0,art_gallery


In [16]:
Df.fillna(0, inplace=True)

dict={}

for item in Acty:
    item_Df = Df[Df["Type"]==item].reset_index()
    item_Df = item_Df.drop("index",1)       
    dict[item] = item_Df
    print(type(dict[item]))
    print(dict[item][["Place","Rating"]])
    print(dict[item]["Place"]) 


<class 'pandas.core.frame.DataFrame'>
            Place          Rating
0  Not Available   Not Available 
0    Not Available 
Name: Place, dtype: object
<class 'pandas.core.frame.DataFrame'>
            Place          Rating
0  Not Available   Not Available 
0    Not Available 
Name: Place, dtype: object
<class 'pandas.core.frame.DataFrame'>
                                      Place Rating
0          Sayreville Recreation Department    4.3
1               Sayreville Parks Department    4.3
2                         War Memorial Park    NaN
3        Blacktown International Sportspark    4.3
4   Boro Field MacArthur Ave. Sayerville NJ    NaN
5               Cannon Park Shopping Centre    3.9
6                             Causeway Park    4.8
7              South River Community Garden    NaN
8              Ken Buchanan Riverfront Park    NaN
9                 Bissett's Recreation Area    NaN
10                            Pacer's Field    NaN
11                              Bailey Park 

In [48]:
x = "soni"
x.upper()

'SONI'

In [49]:
x

'soni'